In [5]:
import cv2
import numpy as np
import face_detection.face_detection as face_detection
import os

count = 0
flag = None

def pixel_to_temperature(pixel):
    temp_min = 80
    temp_max = 110
    pixel_max = 255
    pixel_min = 0
    temp_range = temp_max-temp_min
    temp = (((pixel-pixel_min)*temp_range)/(pixel_max-pixel_min))+temp_min + 4
    return temp

def thermal_surv():
    global count,flag
    face_detector = face_detection.FaceDetector()
    cap = cv2.VideoCapture(0)

    i= 1
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:

            output = frame
            faces = face_detector.detect_with_no_confidence(frame)

            if faces ==[]:
                face = False
                flag = 0
            else: 
                face = True
            
            for (x1,y1,x2,y2) in faces:
                if len(faces)>1:
                    count+=1
                roi = output[y1:y2, x1:x2]
                try:
                    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                except Exception as e:
                    print(e)
                    continue

                try:
                    if face == True:
                        if flag ==0:
                            count+=len(faces)
                            flag = 1
                                

                except ValueError:
                    continue

                # Mean of only those pixels which are in blocks and not the whole rectangle selected
                mean = pixel_to_temperature(np.mean(roi_gray))

                # Colors for rectangles and textmin_area
                temperature = round(mean, 2)
                color = (0, 255, 0) if temperature < 100 else (0, 0, 255)
                

                # Draw rectangles for visualisation
                output = cv2.rectangle(output, (x1, y1), (x2, y2), color, 2)
                cv2.putText(output, "{} F".format(temperature), (x1, y1-5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)  

                if temperature > 100:
                    try:
                        os.mkdir('persons/person'+str(count))
                    except FileExistsError:
                        pass
                    while(face is True):
                        face = frame[y1+2:y2-1,x1+2:x2-1]
                        cv2.imwrite('persons/person'+str(count)+'/person'+str(count)+'_face'+str(i)+'.jpg', face)
                        print("image captured")
                        i+=1
                
            cv2.imshow('Thermal-Surveillance', output)
            # out.write(output)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        else:
            break

    cap.release()
    # out.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    # whole_frame()
    thermal_surv()